<h1 style="padding-top: 25px;padding-bottom: 25px;text-align: left; padding-left: 10px; background-color: #DDDDDD; 
    color: black;"> <img style="float: left; padding-right: 10px; width: 45px" src="https://raw.githubusercontent.com/Harvard-IACS/2018-CS109A/master/content/styles/iacs.png"> CS1090A Introduction to Data Science </h1>

## Lecture 3: Introduction to Pandas

**Harvard University**<br/>
**Fall 2025**<br/>
**Instructors**: Pavlos Protopapas, Kevin Rader</br>
**Preceptor**: Chris Gumb


<hr style='height:2px'>

### Outline
In this class we'll get familiar with the two Pandas data structures, the `Series` and the `DataFrame`, and the various methods for manipulating the data they contain.

We'll start by simply understanding the data structure objects themselves, constructing them from more familiar Python data structures like lists and dictionaries. We'll then motivate subsequent examples by asking questions about the [student survey](https://forms.gle/zZ487CpYfPstrDui7) data which we'll load into a DataFrame from a CSV.

A very simple markdown scaffold exists with some guiding questions, but students are encouraged to ask questions about how to accomplish different goals with Pandas. All code will be written on the spot (🤞), but you'll have access to a completed notebook on Ed under "solutions" if you want to peek.

### Resources
- There is additional Pandas material in the [Bedrock Data Science](https://edstem.org/us/courses/83082/lessons/147401/slides/851872) and [Bedrock Codecasts & Tutorials](https://edstem.org/us/courses/83082/lessons/147410/slides/851967)sections on Ed
- Pandas has their own quick start guide, [Pandas in 10 Minutes](https://pandas.pydata.org/pandas-docs/stable/user_guide/10min.html) (though that seems like a gross underestimate)
- Make friends with the [Pandas documentation](https://pandas.pydata.org/pandas-docs/stable/index.html)
  
### Pandas Data Structures

#### Series
You can think of a Series as a kind of wrapper for a numpy array which provides some helpful attributes and methods.
- Constructing a Series from Python lists.
- Demonstrate attributes like `values`, `index`, `name`, `dtype`.
- Some basic methods like `head()`, `tail()`, `describe()`.

#### DataFrame
- How to construct from Python lists, dictionaries, and Series.
- Attributes like `shape`, `columns`, `index`, `values`.
- Methods like `head()`, `tail()`, `info()`, `describe()`.
- Combining DataFrames with `pd.concatenate()` and `pd.merge()`

A DataFrame is basically just a bunch of Series objects bound together by common indices.

A common way to built a DataFrame is to use a list of dictionaries. Each dictionary is a row and the key-value pairs designate which value appears in which Series (column). For example:

### Data Loading and Inspection
- Use `pd.read_csv()` to load data.
- Initial inspection using `shape`, `head()`, `tail()`, `columns`, `dtypes`, and `describe()`.

Read the survey CSV into a Pandas DataFrame

Take a quick look at a few rows.

How many rows and columns are there?

What are all the columns?

Can we get any summary statistics from the columns?

What are the in data types of each columns?

### Data Cleaning

#### Do we need all of the columns? Are the column names useable? Do we need to change data types? Do we have missing values?

- `drop` (for rows or columns)
- `apply` arbitrary functions applied to a column(s)
- `astype` to convert column dtype
- `str` methods on columns
- `info()` and `isna` for missing values.
- Discuss `NaN` and methods like `fillna()`, `dropna()`.

The column names are too long to be practical. We should rename them.

In [0]:
cols = [
    "timestamp","program","jupyter","python_exp","pandas_skill","os","dark_mode",
    "languages","continents","dob","wake_time","sleep_time","fav_season",
    "caffeine","pet","fav_movie","fav_genres","hobbies","hw0"
]

We won't use the timestamp, so **we should drop it**.

`dark_mode` is a boolean (True/False) variable. Making the dtype reflect this will save time later.

Converting categorical variables like `os` to the `category` dtype can have many advantages. In large datasets this can mean increased speed and memory efficiency. It can also enforce that a variable only take on a perscribed set of values.

The `category` dtype is also useful to impose an odering on variables that are not strictly numeric. We call such variables "ordinal."

This is the case for the `python_exp` variable. Let's convert that column now and specify a sensible order with the help of `pandas.api.types.CategoricalDtype`.

In [0]:
experience_order = pd.CategoricalDtype(categories=['Less than 1 year', '1-2 years', '2-4 years', '4+ years'], ordered=True)
df['python_experience'] = df.python_exp.astype(experience_order)
df.python_exp

**Text Normalization**

For columns whose values were manually typed by the user, upper- and lowercase may not have been use constentily across all entries. For this reason it is usually advised to convert all string columns to lowercase.

In [0]:
# Here's one way to do this
str_cols = df.columns[df.dtypes == 'object']
for c in str_cols:
    df[c] = df[c].str.lower()
df.head()

Do we have any **duplicate rows**?

Do any columns have **missing values**?

How many are missing in each?

Let's inspect the rows with more than a single missing value.

For any students in a bacherlor's program whose `dob` is NA (there is at least one), fill in the mean `dob` of all other bacheloar's students who *do* have a reported age.

What rows would we have remaining if we dropped all rows a missing value for `hobbies`?

### Filtering, Subsetting, Sorting, and Counting

- Boolean indexing.
- Using `.loc[]` and `.iloc[]`
- `sort_values`

Among the students who prefer coding in dark mode, who is the youngest?

### Starting to Ask More Complex Questions
- `unique()`
- `value_counts()`
- `replace()`
- `plot()`
- `explode()`
- `to_numeric()`

How many unique programs are represented?

How many students are in each program?

Some of the "other" program options added by form users seem to refer to the same thing but are just written differently. Consider the Extension School's gratuate certificate for example.

Can we do something to combine these different entries which in fact refer to the same idea?

In [0]:
df['program'] = df.program.replace(r".*certificate.*", "graduate certificate [extension school]", regex=True)
df.program.value_counts()

Can we visualize these new counts?

Can we do something similar for `python_experience` and `pandas_skill`, but this time the proportion rather than count?

What are all the languages CS1090A students speak?

In [0]:
df.languages.str.split(', ').explode().replace(r'.*mandarin.*|madarin|(chinese$)', 'chinese (mandarin)', regex=True).unique().tolist()

**What is the greatest number of languages spoken by a CS109A student?**

**What is the average number of languages spoken by students in programs with at least 2 representatives?**(ideally, sorted)

### Data Aggregation and Grouping

- `groupby()`
- `agg()`
- Aggregation functions like `mean()`, `count()`, `sum()`.

We first need to find which are the big programs to filter the df.

In [0]:
program_counts = df.program.value_counts()
big_programs = program_counts[program_counts >= 2].index
df[df.program.isin(big_programs)].groupby('program')['num_languages'].mean().sort_values()

Does dark mode preference vary by operating system?

`pd.crosstab` helps here.

In [0]:
dark_mode_os_crosstab = pd.crosstab(df['os'], df['dark_mode'])
dark_mode_os_crosstab

What the distribution of student ages?

In [0]:
# Just make sure dob is a datetime (invalid -> nat)
df["dob"] = pd.to_datetime(df["dob"], errors="coerce")

# Compute age in years (floor). Result is nullable integer (Int64).
now = pd.Timestamp.now()
age_years = np.floor((now - df["dob"]).dt.days / 365.25).astype("Int64")

df.loc[:, "age"] = age_years
df.age

Some stats..

And a plot!

**Bar plot of mean Pandas skill by OS type**

In [0]:
df.groupby('os', observed=False)['pandas_skill'].mean().plot(kind='bar', title = 'Pandas Skill');

What about something related to the HW0 emoji valued column?

In [0]:
emoji_order = pd.CategoricalDtype(categories=['🍰', '😎', '🧐', '😅', '😱', '☠️'], ordered=True)

df['hw0'] = df['hw0'].astype(emoji_order)

emoji_counts = df.groupby(['python_experience', 'hw0'], observed=False).size().unstack(fill_value=0)

emoji_proportions = emoji_counts.div(emoji_counts.sum(axis=1), axis=0)

emoji_proportions

### Writing to Files

#### You can save your cleaned dataset to a new CSV file
- `to_csv()`
- `index` argument

🌈 **The End**